In [41]:
import pandas as pd
from sqlalchemy import create_engine
from tabulate import tabulate

# connection details
server = "jarbdatabase.database.windows.net,1433"
database = "Azure_EmployeeDB"
username = "johan.robelto.bayona@johanrobeltobayonaoutlook.onmicrosoft.com"
password = "Outnano92"

# IMPORTANT: use the proper ODBC driver installed on your machine
driver = "ODBC Driver 17 for SQL Server"

# create SQLAlchemy engine string with Active Directory Interactive auth
conn_str = (
    f"mssql+pyodbc://{username}@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
    f"&Authentication=ActiveDirectoryInteractive"
    f"&Encrypt=yes"
    f"&TrustServerCertificate=no"
)

# create engine
engine = create_engine(conn_str)

# now this is fully supported by pandas
query = "SELECT TOP 10 * FROM [SalesLT].[Customer]"
df = pd.read_sql(query, engine)
df_sql = df[df["CustomerID"] == 1]
print(tabulate(df_sql, headers = 'keys', tablefmt = 'psql'))


+----+--------------+-------------+---------+-------------+--------------+------------+----------+---------------+-------------------------+------------------------------+--------------+----------------------------------------------+----------------+--------------------------------------+---------------------+
|    |   CustomerID | NameStyle   | Title   | FirstName   | MiddleName   | LastName   | Suffix   | CompanyName   | SalesPerson             | EmailAddress                 | Phone        | PasswordHash                                 | PasswordSalt   | rowguid                              | ModifiedDate        |
|----+--------------+-------------+---------+-------------+--------------+------------+----------+---------------+-------------------------+------------------------------+--------------+----------------------------------------------+----------------+--------------------------------------+---------------------|
|  0 |            1 | False       | Mr.     | Orlando     | N.  

In [40]:
import pandas as pd
from tabulate import tabulate
df = pd.read_csv("customers_pipe.csv",sep='|',header=0,engine='python')
df_csv = pd.DataFrame(df)
df_csv = df[df["CustomerID"] == 1]
print(tabulate(df_csv, headers = 'keys', tablefmt = 'psql'))

+----+--------------+-------------+---------+-------------+--------------+------------+----------+---------------+-------------------------+-------------------+----------------+------------------------------------------------------------------+----------------+--------------------------------------+---------------------+
|    |   CustomerID | NameStyle   | Title   | FirstName   | MiddleName   | LastName   | Suffix   | CompanyName   | SalesPerson             | EmailAddress      | Phone          | PasswordHash                                                     | PasswordSalt   | rowguid                              | ModifiedDate        |
|----+--------------+-------------+---------+-------------+--------------+------------+----------+---------------+-------------------------+-------------------+----------------+------------------------------------------------------------------+----------------+--------------------------------------+---------------------|
|  0 |            1 | True     

In [43]:
comparison = df_csv.equals(df_sql)
print("DataFrames are identical:", comparison)

DataFrames are identical: False


In [44]:
compare_cols = ['CustomerID', 'FirstName', 'LastName']

differences = df_csv[compare_cols].merge(
    df[compare_cols],
    on='CustomerID',
    suffixes=('_csv', '_sql')
)

differences = differences[differences['FirstName_csv'] != differences['FirstName_sql']]
print(differences)


   CustomerID FirstName_csv LastName_csv FirstName_sql LastName_sql
0           1       Michael        Jones       Orlando          Gee


In [52]:
import pandas as pd
from tabulate import tabulate
df = pd.read_csv("customers_pipe.csv",sep='|',header=0,engine='python')
df_csv = pd.DataFrame(df)
df_csv = df[df["CustomerID"] == 1]
#print(tabulate(filtered_df, headers = 'keys', tablefmt = 'psql'))
merged = df_csv.merge(df_sql, on="CustomerID", suffixes=("_csv", "_sql"), how="outer", indicator=True)
# Show differences
diffs = merged[merged['_merge'] != 'both']
print(diffs)

Empty DataFrame
Columns: [CustomerID, NameStyle_csv, Title_csv, FirstName_csv, MiddleName_csv, LastName_csv, Suffix_csv, CompanyName_csv, SalesPerson_csv, EmailAddress_csv, Phone_csv, PasswordHash_csv, PasswordSalt_csv, rowguid_csv, ModifiedDate_csv, NameStyle_sql, Title_sql, FirstName_sql, MiddleName_sql, LastName_sql, Suffix_sql, CompanyName_sql, SalesPerson_sql, EmailAddress_sql, Phone_sql, PasswordHash_sql, PasswordSalt_sql, rowguid_sql, ModifiedDate_sql, _merge]
Index: []

[0 rows x 30 columns]


In [55]:
common_cols = list(set(df_csv.columns).intersection(df_sql.columns))

csv_df = df_csv[common_cols]
sql_df = df_sql[common_cols]

csv_df = csv_df.sort_values(by="CustomerID").reset_index(drop=True)
sql_df = sql_df.sort_values(by="CustomerID").reset_index(drop=True)

# Merge both datasets
merged = csv_df.merge(sql_df, on="CustomerID", suffixes=('_csv', '_sql'), how='outer', indicator=True)

# Find missing in SQL
missing_in_sql = merged[merged['_merge'] == 'left_only']

# Find missing in CSV
missing_in_csv = merged[merged['_merge'] == 'right_only']

# Find potential mismatches
common = merged[merged['_merge'] == 'both']

mismatched = []
for col in common_cols:
    if col != 'CustomerID':
        diff = common[common[f"{col}_csv"] != common[f"{col}_sql"]]
        if not diff.empty:
            mismatched.append((col, diff[['CustomerID', f'{col}_csv', f'{col}_sql']]))
            
            
print("✅ Records missing in SQL:")
print(missing_in_sql)

print("\n❌ Records missing in CSV:")
print(missing_in_csv)

print("\n⚠️ Column Mismatches:")
for col, df in mismatched:
    print(f"\nColumn: {col}")
    #print(df)
    print(tabulate(df, headers = 'keys', tablefmt = 'psql'))

✅ Records missing in SQL:
Empty DataFrame
Columns: [CompanyName_csv, LastName_csv, EmailAddress_csv, rowguid_csv, Title_csv, MiddleName_csv, PasswordHash_csv, CustomerID, Phone_csv, NameStyle_csv, Suffix_csv, FirstName_csv, SalesPerson_csv, ModifiedDate_csv, PasswordSalt_csv, CompanyName_sql, LastName_sql, EmailAddress_sql, rowguid_sql, Title_sql, MiddleName_sql, PasswordHash_sql, Phone_sql, NameStyle_sql, Suffix_sql, FirstName_sql, SalesPerson_sql, ModifiedDate_sql, PasswordSalt_sql, _merge]
Index: []

[0 rows x 30 columns]

❌ Records missing in CSV:
Empty DataFrame
Columns: [CompanyName_csv, LastName_csv, EmailAddress_csv, rowguid_csv, Title_csv, MiddleName_csv, PasswordHash_csv, CustomerID, Phone_csv, NameStyle_csv, Suffix_csv, FirstName_csv, SalesPerson_csv, ModifiedDate_csv, PasswordSalt_csv, CompanyName_sql, LastName_sql, EmailAddress_sql, rowguid_sql, Title_sql, MiddleName_sql, PasswordHash_sql, Phone_sql, NameStyle_sql, Suffix_sql, FirstName_sql, SalesPerson_sql, ModifiedDate_s